In [230]:
import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib.pyplot as plt

In [231]:
import networkx as nx
import osmnx as ox

ox.settings.log_console=True

map_graph = ox.graph_from_place('Burgos, Spain', network_type='drive')
largest_cc = max(nx.strongly_connected_components(map_graph), key=len)
map_graph = map_graph.subgraph(largest_cc)

METS

In [232]:
def matrix_element_similarity(G,T):
    M=np.zeros((len(T),len(T)))
    G_aux=G.copy()

    for i,track_i in enumerate(T):
        for j,track_j in enumerate(T):
            if i==j:
                continue

            G_aux.add_node("vSource")
            for n in track_i:
                G_aux.add_edge("vSource",n,length=0) #NOT EQUAL TO: n,"vSource" (directed)
            dists=[nx.shortest_path_length(G_aux,"vSource",m, weight='length') for m in track_j]
            G_aux.remove_node("vSource")

            M[i,j]=sum(dists)/len(track_i)
    return M

In [233]:
def matrix_element_similarity_v2(G,T):
    M=np.zeros((len(T),len(T)))
    G_aux=G.copy()

    for i,track_i in enumerate(T):
        for j,track_j in enumerate(T):
            if i==j:
                continue

            G_aux.add_node("vSource")
            G_aux.add_edges_from(("vSource", n, {'length': 0}) for n in track_i)
            dists=[nx.shortest_path_length(G_aux,"vSource",m, weight='length') for m in track_j]
            G_aux.remove_node("vSource")

            M[i,j]=sum(dists)/len(track_i)
    return M

ROW-TS

In [234]:
def row_wise_track_similarity(G,T):
    M=np.zeros((len(T),len(T)))
    G_aux=G.copy()

    for i,track_i in enumerate(T):

        G_aux.add_node("vSource")
        G_aux.add_edges_from(("vSource", n, {'length': 0}) for n in track_i)
        dists=nx.single_source_shortest_path_length(G_aux,"vSource")
        G_aux.remove_node("vSource")

        for j,track_j in enumerate(T):
            if i==j:
                continue
            
            max_dist=max(dists[m] for m in track_j)
                    
            M[i,j]=max_dist
    return M

Performance test

In [235]:
#Choose map
G=map_graph

#Generate random paths to test
from random import choice
n_paths=10
T=[]
for i in range(n_paths):
    orig=choice(list(G.nodes()))
    dest=choice(list(G.nodes()))
    T.append(nx.shortest_path(G, orig, dest, weight='length'))

#Compute tha matrix
import time
t0=time.time()
M_mets=matrix_element_similarity(G,T)
t1=time.time()
print("METS (v1) takes {:.2f}s".format(t1-t0))
t0=time.time()
M_mets=matrix_element_similarity_v2(G,T)
t1=time.time()
print("METS (v2) takes {:.2f}s".format(t1-t0))
t0=time.time()
M_rowts=row_wise_track_similarity(G,T)
t1=time.time()
print("ROW-TS takes {:.2f}s".format(t1-t0))

METS (v1) takes 54.35s
METS (v2) takes 57.07s
ROW-TS takes 0.24s


In [236]:
def symmetrize(M,strategy='min'):
    if strategy=='mean':
        return (1/2)*(M+M.T)
    if strategy=='min':
        return np.minimum(M,M.T)
    if strategy=='max':
        return np.maximum(M,M.T)
    raise KeyError(f'{strategy} is not a valid strategy')

Clustering

In [237]:
from sklearn_extra.cluster import KMedoids

def k_corridors_medoid_summarizer(M,k,T,**kwargs):
    kmedoids = KMedoids(n_clusters=k, metric='precomputed', **kwargs) #For example random_state=0
    kmedoids.fit(M) #M.T is not M!! WHICH ONE TO CHOOSE? :((

    cluster_labels = kmedoids.labels_
    medoid_indices = kmedoids.medoid_indices_
    d=kmedoids.inertia_

    k_corridors=[T[i] for i in medoid_indices]
    return k_corridors,medoid_indices,cluster_labels

Track converters from list of edges and lists of nodes

In [238]:
def edges_to_nodes(track):
    return [edge[0] for edge in track]

def nodes_to_edges(track):
    return [(track[i],track[i+1]) for i in range(len(track)-1)]

Metrics definitions

Pointwise metrics

In [239]:
def pointwise_absolute_intersect(track_i,track_j):
    """
    Returns 1 if there is at least one edge from track_i present in track_j,
    and 0 otherwise
    """

    #Convert list of nodes into list of edges
    set_i=set(nodes_to_edges(track_i))
    set_j=set(nodes_to_edges(track_j))

    #Calculate intesection
    intersect=set_i.intersection(set_j)
    n=len(intersect)
    
    return 0 if n==0 else 1
    

In [240]:
def pointwise_relative_intersect(track_i,track_j):
    """
    Returns what portion of the edges from track_i are present in track_j
    """

    #Convert list of nodes into list of edges
    set_i=set(nodes_to_edges(track_i))
    set_j=set(nodes_to_edges(track_j))

    #Calculate intesection
    intersect=set_i.intersection(set_j)
    n=len(intersect)
    
    return n/len(set_i)

In [241]:
def pointwise_lenght_relative_intersect(track_i,track_j,G):
    """
    Returns what portion of the length of track_i is in the cummulative length of the edges
    also present in track_j
    """

    #Convert list of nodes into list of edges
    set_i=set(nodes_to_edges(track_i))
    set_j=set(nodes_to_edges(track_j))

    #Calculate intersection
    intersect=set_i.intersection(set_j)

    l_intersect=sum([G.get_edge_data(*edge)[0]['length'] for edge in intersect])
    l_i=sum([G.get_edge_data(*edge)[0]['length'] for edge in set_i])


    
    return l_intersect/l_i

'k' metrics (applied to _all_ k corridors at once, not one by one)

In [242]:
def k_absolute_intersect(track_i,k_tracks):
    """
    Returns 1 if there is at least one edge from track_i present in k_tracks,
    and 0 otherwise
    """

    #Convert list of nodes into list of edges
    set_i=set(nodes_to_edges(track_i))

    for track_j in k_tracks:
        set_j=set(nodes_to_edges(track_j))

        #Calculate intesection
        intersect=set_i.intersection(set_j)
        n=len(intersect)
        if n!=1:
            return 1
    
    return 0
    

In [243]:
def k_relative_intersect(track_i,k_tracks):
    """
    Returns what portion of the edges from track_i are present in k_tracks
    """

    #Convert list of nodes into list of edges
    set_i=set(nodes_to_edges(track_i))
    k_edges=np.concatenate([nodes_to_edges(track_j) for track_j in k_tracks])
    set_j=set([(n1,n2) for n1,n2 in k_edges])

    #Calculate intesection
    intersect=set_i.intersection(set_j)
    n=len(intersect)
    
    return n/len(set_i)

In [244]:
def k_lenght_relative_intersect(track_i,k_tracks,G):
    """
    Returns what portion of the length of track_i is in the cummulative length of the edges
    also present in k_tracks
    """

    #Convert list of nodes into list of edges
    set_i=set(nodes_to_edges(track_i))
    k_edges=np.concatenate([nodes_to_edges(track_j) for track_j in k_tracks])
    set_j=set([(n1,n2) for n1,n2 in k_edges])

    #Calculate intersection
    intersect=set_i.intersection(set_j)

    l_intersect=sum([G.get_edge_data(*edge)[0]['length'] for edge in intersect])
    l_i=sum([G.get_edge_data(*edge)[0]['length'] for edge in set_i])


    
    return l_intersect/l_i

Metrics evaluation

In [245]:
def evaluate_pointwise_metric(metric_fun,T,k_corridors,**kwargs):
    """
    Returs the cummulative value (normalized by the number of tracks) of the 
    selected metric for the selected k corridors.
    T: A list of tracks to be evaluated (if k_corridors are included they will also be evaluated)
    k_corridors: A list of k selected corridors
    metric_fun: The metric function to evaluate as metric_fun(T[i],k_corridors[j],**kwargs)
    """

    result=0
    for track_i in T:
        for track_j in k_corridors:
            result+=metric_fun(track_i,track_j,**kwargs)

    result/=(len(T)*len(k_corridors))

    return result

In [246]:
def evaluate_k_metric(metric_fun,T,k_corridors,**kwargs):
    """
    Returs the cummulative value (normalized by the number of tracks) of the 
    selected metric for the selected k corridors.
    T: A list of tracks to be evaluated (if k_corridors are included they will also be evaluated)
    k_corridors: A list of k selected corridors
    metric_fun: The metric function to evaluate as metric_fun(T[i],k_corridors[j],**kwargs)
    """

    result=0
    for track_i in T:
        result+=metric_fun(track_i,k_corridors,**kwargs)

    result/=len(T)

    return result

In [247]:
# def evaluate_M(G,T,cluster_labels,k_index,sym_strategy=None):
#     M=row_wise_track_similarity(G,T)
#     if sym_strategy:
#         M=symmetrize(M,strategy=sym_strategy)

#     d=0
#     for i in range(len(k_index)):
#         d+=M[cluster_labels==i,k_index[i]].sum() #TODO: Not always symetric! this or the other way?
#     return d

def evaluate_M(G,T,cluster_labels,k_index,sym_strategy=None):
    M=row_wise_track_similarity(G,T)
    if sym_strategy:
        M=symmetrize(M,strategy=sym_strategy)

    d=np.min(M[:,k_index],axis=1).sum()
    return d

def M_similarity(G,T,cluster_labels,k_index,include_corridors=False,ponderated=True,ponderation_k=1):
    '''
    This metric returns a metric based on the average similarity distance from
    the tracks to the corridors.
    If include_corridors, the average is made taking into consideration the corridors
    as tracks too, otherwise the average is made using only the rest of the tracks.
    
    If ponderated, the result is ponderated so that the metric ranges (0,1],
    where 1 means all tracks are corridors and 0 tracks are at an infinite distance from the corridors.
    The ponderation is k/(k+d), where k is the ponderation parameter that defines at which d
    the ponderated score is equal to 1/2
    '''
    d=evaluate_M(G,T,cluster_labels,k_index)
    n=len(T)
    if not include_corridors:
        n-=len(k_index)
    d/=n
    if ponderated:
        return ponderation_k/(ponderation_k+d)
    return d

Demo:

In [250]:
#Define sizes
n_paths=100
n_clusters=5

#Choose map
G=map_graph

#Generate random paths to test
from random import choice

T=[]
for i in range(n_paths):
    orig=choice(list(G.nodes()))
    dest=choice(list(G.nodes()))
    T.append(nx.shortest_path(G, orig, dest, weight='length'))

In [251]:
#Compute the similarity matrix
M_rowts=row_wise_track_similarity(G,T)


k_corridors,k_index,cluster_labels=k_corridors_medoid_summarizer(M_rowts,n_clusters,T,random_state=0)

print("For ROW-TS:")

for sym_strategy in ["mean","min","max"]:
    #Symmetrize M
    M=symmetrize(M_rowts,strategy=sym_strategy)
    #Cluster in k corridors
    k_corridors,k_index,cluster_labels=k_corridors_medoid_summarizer(M,n_clusters,T,random_state=0)

    #Test performance

    print(f'\n\n------------------\n\t{sym_strategy}\t\n------------------')

    print("\nPointwise metrics:\n------------------")

    abs_inter=evaluate_pointwise_metric(pointwise_absolute_intersect,T,k_corridors)
    print("Absolute intersection: {:.2f}".format(abs_inter))

    rel_inter=evaluate_pointwise_metric(pointwise_relative_intersect,T,k_corridors)
    print("Relative intersection: {:.4f}".format(rel_inter))

    l_rel_inter=evaluate_pointwise_metric(pointwise_lenght_relative_intersect,T,k_corridors,G=G)
    print("Length intersection: {:.4f}".format(l_rel_inter))

    print("\nk-set metrics:\n--------------")

    abs_inter=evaluate_k_metric(k_absolute_intersect,T,k_corridors)
    print("Absolute intersection: {:.2f}".format(abs_inter))

    rel_inter=evaluate_k_metric(k_relative_intersect,T,k_corridors)
    print("Relative intersection: {:.2f}".format(rel_inter))

    l_rel_inter=evaluate_k_metric(k_lenght_relative_intersect,T,k_corridors,G=G)
    print("Length intersection: {:.2f}".format(l_rel_inter))

    print("\nM metrics:\n----------")

    m_sim=M_similarity(G,T,cluster_labels,k_index,ponderated=False,include_corridors=True)
    print("M average distance: {:.0f}m".format(m_sim))

    m_sim=M_similarity(G,T,cluster_labels,k_index,ponderated=True,include_corridors=True)
    print("Ponderated M similarity: {:.4f}".format(m_sim))

For ROW-TS:


------------------
	mean	
------------------

Pointwise metrics:
------------------
Absolute intersection: 0.24
Relative intersection: 0.0316
Length intersection: 0.0297

k-set metrics:
--------------
Absolute intersection: 1.00
Relative intersection: 0.15
Length intersection: 0.15

M metrics:
----------
M average distance: 14m
Ponderated M similarity: 0.0673


------------------
	min	
------------------

Pointwise metrics:
------------------
Absolute intersection: 0.28
Relative intersection: 0.0587
Length intersection: 0.0607

k-set metrics:
--------------
Absolute intersection: 1.00
Relative intersection: 0.27
Length intersection: 0.29

M metrics:
----------
M average distance: 30m
Ponderated M similarity: 0.0319


------------------
	max	
------------------

Pointwise metrics:
------------------
Absolute intersection: 0.32
Relative intersection: 0.0354
Length intersection: 0.0307

k-set metrics:
--------------
Absolute intersection: 1.00
Relative intersection: 0.13
Len

Test best symetrization strategy

In [252]:
n_paths_clusters_list=[(25,5),(50,5),(100,5)]
n_experiments=10

G=map_graph

strategies=[None,"mean","min","max"]
results={s:[] for s in strategies}

from random import choice
import warnings
warnings.filterwarnings('ignore')

for n_paths,n_clusters in n_paths_clusters_list:
    print(f"\n\nIntermediate result: {n_paths} paths, {n_clusters} clusters")
    iter_results={s:[] for s in strategies}
    for strategy in strategies:
        for i in range(n_experiments):
            T=[]
            for i in range(n_paths):
                orig=choice(list(G.nodes()))
                dest=choice(list(G.nodes()))
                T.append(nx.shortest_path(G, orig, dest, weight='length'))

            M=row_wise_track_similarity(G,T)
            if strategy:
                M=symmetrize(M,strategy=strategy)
            k_corridors,k_index,cluster_labels=k_corridors_medoid_summarizer(M,n_clusters,T)
            try:
                l_rel_inter=evaluate_k_metric(k_lenght_relative_intersect,T,k_corridors,G=G)
            except ZeroDivisionError:
                continue

            iter_results[strategy].append(l_rel_inter)
    for s,v in iter_results.items():
        print(f'{s}: {np.mean(v):.3f}±{np.std(v):.3f}',end="\t")
        results[s]+=v
print("\n\nGlobal results:")
for s,v in results.items():
        print(f'{s}: {np.mean(v):.3f}±{np.std(v):.3f}',end="\t")
warnings.filterwarnings('default')



Intermediate result: 25 paths, 5 clusters
None: 0.257±0.025	mean: 0.301±0.031	min: 0.323±0.040	max: 0.308±0.040	

Intermediate result: 50 paths, 5 clusters
None: 0.134±0.020	mean: 0.213±0.027	min: 0.233±0.038	max: 0.217±0.025	

Intermediate result: 100 paths, 5 clusters
None: 0.090±0.017	mean: 0.165±0.040	min: 0.215±0.059	max: 0.164±0.024	

Global results:
None: 0.161±0.074	mean: 0.226±0.066	min: 0.257±0.066	max: 0.230±0.067	